# Chapter 7. Advanced Text Generation Techniques and Tools

!["7.1"](img/7.1.png)

In [1]:
from langchain import LlamaCpp

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="./Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

llm.invoke("Hi! My name is Maarten. What is 1 + 1?")


llama_init_from_model: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


''

### Chain

In [9]:
from langchain import PromptTemplate

# Create a prompt template with the "input_prompt" variable
template = """<|user|>
{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

In [40]:
basic_chain = prompt | llm

# formatted_prompt = prompt.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})
# output = llm.invoke(formatted_prompt)
# output

In [43]:
# Use the chain
basic_chain.invoke(
    {
        "input_prompt": "Hi! My name is Maarten. What is 1 + 1?",
    }
)

' Hello Maarten! The answer to 1 + 1 is 2.'

### A chain with multiple prompts

!["7.8"](img/7.8.png)

In [23]:
from langchain import LLMChain

# Create a chain for the title of our story
template = """<|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

In [24]:
title.invoke({"summary": "a girl that lost her mother"})

{'summary': 'a girl that lost her mother',
 'title': ' "The Unfading Embrace: A Journey Through Grief"'}

In [32]:
# Create a chain for the character description using the summary and title
template = """<|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title"]
)
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [35]:
# Create a chain for the story using the summary, title, and character description
template = """<|user|>
Create a story about {summary} with the title {title}. The main charachter is: {character}. Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title", "character"]
)
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [36]:
# Combine all three components to create the full chain
llm_chain = title | character | story

llm_chain.invoke("a girl that lost her mother")

{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of a Lost Mother: A Journey Through Grief"',
 'character': " The main character, Lily, is a resilient yet fragile young girl struggling to cope with the overwhelming grief of losing her beloved mother. Throughout the story, she embarks on an emotional journey to reconcile her pain and find solace in cherished memories and newfound strength.\n\nLily is depicted as a sensitive and empathetic child who yearns for her mother's comforting presence, making it challenging for her to navigate the turmoil that grief brings upon her life. As she grows older, however, Lily finds herself slowly adapting, displaying an inner strength borne from her desire to honor her late mother and keep their bond alive in every heartbeat of her life.",
 'story': " Whispers of a Lost Mother: A Journey Through Grief\n\nLily, an empathetic and sensitive young girl, found herself engulfed in the suffocating waves of grief after losing her beloved mothe

## Memory: Helping LLMs to remember conversation

In [41]:
# Let's give the LLM our name
basic_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})

" Hello Maarten! 1 + 1 equals 2. It's a basic arithmetic addition problem."

In [42]:
# Next, we ask the LLM to reproduce the name
basic_chain.invoke({"input_prompt": "What is my name?"})

" I'm unable to determine your name without more context. Privacy guidelines prevent me from knowing personal information about individuals. If you need help with something specific, feel free to ask!"

In [52]:
# Create an updated prompt template to include a chat history
template = """<|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

### ConversationBufferMemory

In [45]:
from langchain.memory import ConversationBufferMemory

# Define the type of Memory we will use
memory = ConversationBufferMemory(memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

C:\Users\IdeaPad\AppData\Local\Temp\ipykernel_31908\1531358956.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [46]:
# Generate a conversation and ask a basic question
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})

c:\Users\IdeaPad\anaconda3\envs\pytorch\Lib\site-packages\llama_cpp\llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'Hi! My name is Maarten. What is 1 + 1?',
 'chat_history': '',
 'text': " The answer to 1 + 1 is 2. It's a basic arithmetic operation where you add one unit to another, resulting in two units."}

In [47]:
# Does the LLM remember the name we gave it?
llm_chain.invoke({"input_prompt": "What is my name?"})

c:\Users\IdeaPad\anaconda3\envs\pytorch\Lib\site-packages\llama_cpp\llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Maarten. What is 1 + 1?\nAI:  The answer to 1 + 1 is 2. It's a basic arithmetic operation where you add one unit to another, resulting in two units.",
 'text': ' Your name is Maarten.\n\nAs for the math question, 1 + 1 equals 2.'}

### Windowed Conversation Buffer

In [53]:
from langchain.memory import ConversationBufferWindowMemory

# Retain only the last 2 conversations in memory
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [54]:
# Ask two questions and generate two conversations in its memory
llm_chain.invoke({"input_prompt":"Hi! My name is Maarten and I am 33 years old. What is 1 + 1?"})
llm_chain.invoke({"input_prompt":"What is 3 + 3?"})

{'input_prompt': 'What is 3 + 3?',
 'chat_history': "Human: Hi! My name is Maarten and I am 33 years old. What is 1 + 1?\nAI:  The answer to 1 + 1 is 2. It seems you're engaging in a friendly conversation! How can I assist you further? Perhaps with some math or other information related to your interests or age-related topics?\n\nRemember, it's always great to share bits of personal information, but also happy to help on any specific questions you might have.",
 'text': " The answer to 3 + 3 is 6. How can I further assist you today, Maarten? Is there anything specific you'd like to know or discuss?\n\nRemember, I'm here to help with any questions you have!"}

In [55]:
# Check whether it knows the name we gave it
llm_chain.invoke({"input_prompt":"What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Maarten and I am 33 years old. What is 1 + 1?\nAI:  The answer to 1 + 1 is 2. It seems you're engaging in a friendly conversation! How can I assist you further? Perhaps with some math or other information related to your interests or age-related topics?\n\nRemember, it's always great to share bits of personal information, but also happy to help on any specific questions you might have.\nHuman: What is 3 + 3?\nAI:  The answer to 3 + 3 is 6. How can I further assist you today, Maarten? Is there anything specific you'd like to know or discuss?\n\nRemember, I'm here to help with any questions you have!",
 'text': " Your name is Maarten, as you mentioned at the beginning of our conversation.\nHere's the answer to your question: 3 + 3 equals 6. If there's anything else I can help you with, feel free to ask!"}

In [56]:
# Check whether it knows the age we gave it
llm_chain.invoke({"input_prompt":"What is my age?"})

{'input_prompt': 'What is my age?',
 'chat_history': "Human: What is 3 + 3?\nAI:  The answer to 3 + 3 is 6. How can I further assist you today, Maarten? Is there anything specific you'd like to know or discuss?\n\nRemember, I'm here to help with any questions you have!\nHuman: What is my name?\nAI:  Your name is Maarten, as you mentioned at the beginning of our conversation.\nHere's the answer to your question: 3 + 3 equals 6. If there's anything else I can help you with, feel free to ask!",
 'text': " I'm sorry, but as an AI, I don't have access to personal data such as your age unless you provide it during our conversation. Privacy is important, so let's focus on general questions or topics you might need assistance with!"}

### Conversation Summary

In [57]:
# Create a summary prompt template
summary_prompt_template = """<|user|>Summarize the conversations and update with the new lines.

Current summary:
{summary}

new lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template
)

In [58]:
from langchain.memory import ConversationSummaryMemory

# Define the type of memory we will use
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)

# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

C:\Users\IdeaPad\AppData\Local\Temp\ipykernel_31908\4216441337.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


In [59]:
# Generate a conversation and ask for the name
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': ' **Updated summary:**  \n\nIn our latest exchange, Maarten introduced himself and posed a classic arithmetic question in English - what is 1 + 1. Responding with enthusiasm and an educational twist, the AI, MathGenius3000, confirmed that the answer to this basic addition problem is indeed two. To add more flavor to our interaction, MathGenius3000 engaged in a role-play by presenting the same information using French language along with an analogy related to shoes and painting, providing both linguistic diversity and a creative approach to explaining the concept of addition.',
 'text': ' Your name has not been mentioned in our current conversation, but you can refer to me as MathGenius3000. If you have a preferred nickname or alias for this AI, feel free to use it!'}

In [60]:
# Check whether it has summarized everything thus far
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

{'input_prompt': 'What was the first question I asked?',
 'chat_history': ' **Updated Summary:**\n\nIn our latest conversation, the human named Maarten interacted with AI MathGenius3000 by asking a basic arithmetic question in English ("What is 1 + 1?") which was answered correctly as two. Additionally, MathGenius3000 demonstrated linguistic diversity and creativity by answering the same question in French using an analogy involving shoes and painting to explain addition. Further into the conversation, when asked for his name, MathGenius3000 clarified its identity as a neutral AI entity but offered flexibility for user-preferred names or nicknames.',
 'text': ' The first question you asked was, "What is 1 + 1?"'}

In [61]:
# Check what the summary is thus far
memory.load_memory_variables({})

{'chat_history': ' **Updated Summary:**\nDuring the conversation, human Maarten inquired MathGenius3000 with a basic arithmetic question "What is 1 + 1?" The AI correctly provided the answer as two. It further showcased linguistic diversity and creativity by explaining addition through an analogy involving shoes and painting in French ("Un chaussure avec une peinture, cela fait deux!"). When asked for its name, MathGenius3000 identified itself as a neutral AI but expressed willingness to be addressed by alternative names or nicknames if preferred. Maarten also sought clarification on the initial question he posed, which prompted MathGenius3000 to reaffirm that it was "1 + 1".'}

## Agents: Creating a System of LLMs

!["7.15"](img/7.15.png)

!["7.16"](img/7.16.png)

In [3]:
from google import genai
import os

api_key = os.environ["GEMINI_API_KEY"]
client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.0-flash", contents="Explain how AI works in a few words"
)
print(response.text)

AI learns patterns from data to make predictions or decisions.



In [4]:
response = client.models.generate_content(
    model="gemini-1.5-flash-8b", contents="Explain how AI works in a few words"
)
print(response.text)

AI learns from data to make predictions or decisions.



In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ["GOOGLE_API_KEY"] = os.environ["GEMINI_API_KEY"]

gemini = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-8b",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [10]:
# Create the ReAct template
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [13]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# Prepare tools
tools = load_tools(["llm-math"], llm=gemini)
tools.append(search_tool)

In [14]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent
agent = create_react_agent(gemini, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [15]:
# What is the Price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)



> Entering new AgentExecutor chain...
Question: What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?
Thought: I need to find the current price of a MacBook Pro in USD and then convert it to EUR using the given exchange rate.
Action: duckduck
Action Input: current price of macbook prosnippet: Find the cheapest prices and best deals on current MacBook Pros at AppleInsider., title: Best MacBook Pro Deals for March 2025 | Save up to $1,200, link: https://appleinsider.com/deals/best-macbook-pro-deals, snippet: With power, performance, and premium style, the MacBook Pro represents the pinnacle of Apple's laptop offerings. This comprehensive guide breaks down current MacBook Pro pricing across available sizes, processors, storage capacities, and more. You'll learn how choices at checkout impact final cost, find savings opportunities, and gain insight into Apple's pricing philosophy on its ..., title: How Much Does a Mac

{'input': 'What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?',
 'output': 'A 14-inch M4 MacBook Pro (16GB/512GB) costs $1399 USD.  At an exchange rate of 0.85 EUR per 1 USD, it would cost approximately 1189.15 EUR.'}

In [20]:
gemini = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# Prepare tools
tools = load_tools(["llm-math"], llm=gemini)
tools.append(search_tool)

# Construct the ReAct agent
agent = create_react_agent(gemini, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in KRW if the exchange rate is 1467.60 KRW for 1 USD?"
    }
)



> Entering new AgentExecutor chain...
I need to find the current price of a MacBook Pro in USD and then convert that price to KRW using the given exchange rate.
Action: duckduck
Action Input: "current price of a MacBook Pro"snippet: Apple resellers are hosting a variety of MacBook Pro sales that discount current M4, M4 Pro and M4 Max 14-inch and 16-inch models, in addition to blowout bargains on M3 models. Apple offers two ..., title: Best MacBook Pro Deals for March 2025 | Save up to $1,200 - AppleInsider, link: https://appleinsider.com/deals/best-macbook-pro-deals, snippet: A MacBook Pro cost ranges from just over $1000 to well over $3000. Determining the true cost for the MacBook Pro you need requires close examination of the model options, hardware configurations, and Apple's pricing strategies. Let's examine how Apple prices each MacBook Pro base model and default hardware configuration: MacBook Pro 13-inch, title: How Much Does a MacBook Pro Cost? - The Pricer, link: https://ww

{'input': 'What is the current price of a MacBook Pro in USD? How much would it cost in KRW if the exchange rate is 1467.60 KRW for 1 USD?',
 'output': 'The price of the MacBook Pro is $1699, which is equivalent to 2,493,452.4 KRW at the given exchange rate.'}